***
Supervised training with pairs of noisy and clean images
***

In [1]:
from unet import Unet
from n2v import N2VConfig, N2VDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [2]:
file_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-lowsnr.tif'
gt_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-highsnr.tif'
unet = Unet(n_depth=3, kernel_size=5)
config=N2VConfig([file_path],[gt_path],patch_shape=[2048,2048], patches_per_batch=1, validation_split=0.08)
n2v_data_generator=N2VDataGenerator(config)

In [ ]:
history=unet.train(n2v_data_generator, epochs=40, supervised=True)

In [ ]:
result_supervised=unet.evaluate(n2v_data_generator, save_dir="restored_supervised")
ssim=np.array(result_supervised['ssim'])
psnr=np.array(result_supervised['psnr'])
old_ssim=np.array(result_supervised['old_ssim'])
old_psnr=np.array(result_supervised['old_psnr'])
print("For all the images".center('-',40))
print('duration:',result_supervised['duration'])
print('ssim:',ssim.mean().round(2),'\tpsnr:', psnr.mean().round(2))
print('old_ssim:', old_ssim.mean().round(2),'\told_psnr:', old_psnr.mean().round(2))

print("For the first image".center('-',40))
print('ssim:',ssim[0].round(2),'\tpsnr:', psnr[0].round(2))
print('old_ssim:', old_ssim[0].round(2),'\told_psnr:', old_psnr[0].round(2))

In [ ]:
unet.model.save_weights("model_weights/supervised_ndepth3_kernelsize5.h5")

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']
mae=history.history['mae']
val_mae=history.history['val_mae']

epochs=range(1,1+len(loss))
plt.figure(figsize=(14,7))
plt.subplot(121)
plt.plot(epochs,loss,label='mse')
plt.plot(epochs,val_loss,label='val_mse')
plt.xlabel("epoch");plt.legend()
plt.subplot(122)
plt.plot(epochs,mae,label='mae')
plt.plot(epochs,val_mae,label='val_mae')
plt.xlabel("epoch");plt.legend()
plt.show()

***
Self-supervised training with only noisy images
***

In [ ]:
from unet import Unet
from n2v import N2VConfig, N2VDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
file_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-lowsnr.tif'
gt_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-highsnr.tif'
unet = Unet(n_depth=3, kernel_size=5)
config=N2VConfig([file_path],[gt_path],patch_shape=[256,256],patches_per_batch=32,validation_split=0.08)
n2v_data_generator=N2VDataGenerator(config)

In [ ]:
history=unet.train(n2v_data_generator, epochs=40)

In [ ]:
result=unet.evaluate(n2v_data_generator, save_dir="restored_self_supervised")
ssim=np.array(result['ssim'])
psnr=np.array(result['psnr'])
old_ssim=np.array(result['old_ssim'])
old_psnr=np.array(result['old_psnr'])
print("For all the images".center('-',40))
print('duration:',result['duration'])
print('ssim:',ssim.mean().round(2),'\tpsnr:', psnr.mean().round(2))
print('old_ssim:', old_ssim.mean().round(2),'\told_psnr:', old_psnr.mean().round(2))

print("For the first image".center('-',40))
print('ssim:',ssim[0].round(2),'\tpsnr:', psnr[0].round(2))
print('old_ssim:', old_ssim[0].round(2),'\told_psnr:', old_psnr[0].round(2))

In [ ]:
unet.model.save_weights("model_weights/self_supervised_ndepth3_kernelsize5.h5")

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(1,1+len(loss))
plt.plot(epochs,loss,label='loss')
plt.plot(epochs,val_loss,label='val_loss')
plt.xlabel("epoch")
plt.legend()
plt.show()

***
Self-supervised training on one single image
***

In [3]:
from unet import Unet
from n2v import N2VConfig, N2VDataGenerator
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

In [4]:
file_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-lowsnr.tif'
gt_path=r'C:\Users\bioimaging1\Desktop\Deep learning image\actin\actin-60x-noise1-highsnr.tif'
with Image.open(file_path) as img,Image.open(gt_path) as gt:
    img.save("singleimage.tif")
    gt.save("singleimage_gt.tif")

In [ ]:
file_path="singleimage.tif"
gt_path="singleimage_gt.tif"
unet = Unet(n_depth=3, kernel_size=5)
config=N2VConfig([file_path],[gt_path],patch_shape=[256,256],patches_per_batch=32,validation_split=0)
n2v_data_generator=N2VDataGenerator(config)

In [ ]:
history=unet.train(n2v_data_generator, epochs=40*100)

In [ ]:
result_singleimage=unet.evaluate(n2v_data_generator, save_dir="restored_self_supervised_singleimage")
ssim=np.array(result_singleimage['ssim'])
psnr=np.array(result_singleimage['psnr'])
old_ssim=np.array(result_singleimage['old_ssim'])
old_psnr=np.array(result_singleimage['old_psnr'])
print("For the single image".center('-',40))
print('duration:',result['duration'])
print('ssim:',ssim.mean().round(2),'\tpsnr:', psnr.mean().round(2))
print('old_ssim:', old_ssim.mean().round(2),'\told_psnr:', old_psnr.mean().round(2))

In [ ]:
unet.model.save_weights("model_weights/self_supervised_ndepth3_kernelsize5_singleimage.h5")

In [ ]:
loss=history.history['loss']

epochs=range(1,1+len(loss))
plt.plot(epochs,loss,label='loss')
plt.xlabel("epoch")
plt.legend()
plt.show()


---

In [ ]:
config = N2VConfig(file_paths=["example.tif"],
                   ground_truth_paths=["example_gt.tif"],
                   patch_shape=(64, 64),
                   patches_per_batch=128,
                   data_augmentation=True,
                   epochs=100)
unet = Unet(config, n_depth=2)


In [ ]:
n2v_data_generator = N2VDataGenerator(config)
history=unet.train(n2v_data_generator)

In [ ]:
result=unet.evaluate(n2v_data_generator, divide=1, save_dir="restored_0")
ssim=np.array(result['ssim'])
psnr=np.array(result['psnr'])
old_ssim=np.array(result['old_ssim'])
old_psnr=np.array(result['old_psnr'])
print(result['duration'])
print('ssim:',ssim.mean().round(2),'\tpsnr:', psnr.mean().round(2))
print('old_ssim:', old_ssim.mean().round(2),'\told_psnr:', old_psnr.mean().round(2))

In [ ]:

from os.path import join
dataset_dir = r'C:\Users\bioimaging1\Desktop\Deep learning image\actin'
noisy_inputs = ["actin-20x-noise1-lowsnr.tif",
            # "actin-confocal-lowsnr.tif",
            # "actin-60x-noise1-lowsnr.tif",
            # "membrane-lowsnr.tif",
            # "mito-20x-noise1-lowsnr.tif",
            # "mito-60x-noise1-lowsnr.tif",
            # "mito-confocal-lowsnr.tif",
            # "nucleus-lowsnr.tif",
            # "mito-60x-noise2-lowsnr.tif"
            ]
ground_truth = ["actin-20x-noise1-highsnr.tif",
            # "actin-confocal-highsnr.tif",
            # "actin-60x-noise1-highsnr.tif",
            # "membrane-highsnr.tif",
            # "mito-20x-noise1-highsnr.tif",
            # "mito-60x-noise1-highsnr.tif",
            # "mito-confocal-highsnr.tif",
            # "nucleus-highsnr.tif",
            # "mito-60x-noise2-highsnr.tif"
            ] 
file_paths = [join(dataset_dir, dataset) for dataset in noisy_inputs]
ground_truth_paths = [join(dataset_dir, dataset) for dataset in ground_truth]
config = N2VConfig(file_paths, ground_truth_paths,
                   patch_shape=(128, 128),
                   validation_split=0.1,
                   patches_per_batch=32,
                   data_augmentation=True,
                   epochs=10)

n2v_data_generator = N2VDataGenerator(config)
unet = Unet(config, model_summary=False)

In [ ]:
config = N2VConfig(file_paths, ground_truth_paths,
                   patch_shape=(128, 128),
                   validation_split=0.1,
                   patches_per_batch=32,
                   data_augmentation=True,
                   epochs=20)

n2v_data_generator = N2VDataGenerator(config)
history = unet.train(n2v_data_generator, early_stopping_patience=5, reduce_lr_patience=2                                                                                )
# 可以在Tensorboard查看训练损失和验证损失的变化情况
# cmd: tensorboard --logdir tensorboard 

In [ ]:
# unet.model.load_weights("ckpt/best")
result=unet.evaluate(n2v_data_generator, divide=8, save_dir="restored_images_test1")


In [ ]:
unet.model.load_weights("ckpt/best")
result=unet.evaluate(n2v_data_generator, divide=1, save_dir=None)
ssim=np.array(result['ssim'])
psnr=np.array(result['psnr'])
old_ssim=np.array(result['old_ssim'])
old_psnr=np.array(result['old_psnr'])
print(result['duration'])
print('ssim:',ssim.mean(),'\tpsnr:', psnr.mean())
print('old_ssim:',old_ssim.mean(),'\told_psnr:', old_psnr.mean())

In [ ]:
loss=history.history['loss']
val_loss=history.history['val_loss']

epochs=range(len(loss))
plt.plot(loss,label='loss')
plt.plot(val_loss,label='val_loss')
plt.xlabel("epoch")
plt.legend()
